In [1]:
import numpy as np, os, sys
sys.path.append("..")

import torch
from torch import nn
from torch.nn import functional as F 

from src.utils.synthetic_seqdata import download_data, load_data, sequence_string_to_one_hot
from src.models.deepbind_cnn import BasicCNN
from src.trainer.trainer import Trainer
from src.utils.datasets import DNASequenceDataset

import matplotlib as mpl 
from matplotlib import pyplot as plt 
%matplotlib inline

In [2]:
# get data 
savedir = "./data"
_=download_data(savedir)
Xs, Ys = load_data(savedir=savedir)

--2023-07-26 09:56:41--  https://www.dropbox.com/s/drnyowfdv1lbjz6/train_sequences.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/drnyowfdv1lbjz6/train_sequences.txt [following]
--2023-07-26 09:56:41--  https://www.dropbox.com/s/raw/drnyowfdv1lbjz6/train_sequences.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1939c5715545764b27b92c9e0c.dl.dropboxusercontent.com/cd/0/inline/CAlu-f3Ha8gBYshhFPCzoKsw2WoWvOT_eh6DkA7PBo3cz1BJpbuTMOGVeNukY44fFZ0DLXZvqCkIWzNthBu1ZOfD_1VLoJKusFFLeTMJnGr6qjbsfBrWm3TCW0QP_cXsALU/file# [following]
--2023-07-26 09:56:41--  https://uc1939c5715545764b27b92c9e0c.dl.dropboxusercontent.com/cd/0/inline/CAlu-f3Ha8gBYshhFPCzoKsw2WoWvOT_eh6DkA7PBo3cz1BJpbuTMOGVeNukY44fFZ0DLXZvqCkIWzNthBu1ZOfD_1VLoJKusFFLeT

In [3]:
# set up datasets
datasets = {}
for k in Xs:
    datasets[k] = DNASequenceDataset(sequences=Xs[k], labels=Ys[k], alphabet="ACGT")

# set up dataloaders 
batch_size = 32
loaders = {}
for k, dataset in datasets.items():
    loaders[k] = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
    
# set up the model, lossfn, optimizer, trainer 
learning_rate = 1e-3
model = BasicCNN(input_size=4, output_size=1, kernel_size=9)
lossfn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
trainer = Trainer(model, lossfn, optimizer)

# fit model
nepochs = 50
trainer.fit(loaders['train'], val_loader=loaders['valid'], nepochs=nepochs)

 --- [Epoch   1/50] --- 
loss : 0.7754674158548112:.4f, val_loss : 0.8024956565054636:.4f
 --- [Epoch   2/50] --- 
loss : 0.7159965209764977:.4f, val_loss : 0.7065957235911536:.4f
 --- [Epoch   3/50] --- 
loss : 0.6981169644011754:.4f, val_loss : 0.7008239712033953:.4f
 --- [Epoch   4/50] --- 
loss : 0.6997941958305498:.4f, val_loss : 0.6943972981165326:.4f


KeyboardInterrupt: 

In [ ]:
!rm -rf data